In [1]:
import sys
sys.path.append('/Users/jordi/Desktop/sympy-C2')

# Problema 1

In [2]:
from corbes import Corba
from sympy import sin, cos, pi, symbols, Eq
from sympy import solve, Matrix
x, y, z = symbols('x y z', real=True)
t = Corba.parameter()

In [3]:
eqs = [Eq(x**2 + y**2, 1), Eq(y**2 + z**2, 1)]

In [4]:
r1 = Corba((t,0,2*pi), [cos(t), sin(t), -cos(t)])
r2 = Corba((t,0,2*pi), [cos(t), sin(t), cos(t)])

In [5]:
for r in (r1,r2):
    _,d = r.asdict(x,y,z)
    assert all(e.subs(d).simplify() for e in eqs)

## Punts d'intersecció

In [6]:
s = Corba.parameter('s')
[Eq(l,r) for l,r in zip(r1.point_at(t), r2.point_at(s))]

[cos(_t) == cos(_s), sin(_t) == sin(_s), -cos(_t) == cos(_s)]

In [7]:
ti = (pi/2, 3*pi/2)
for u in ti:
    assert all(Eq(l,r) for l,r in zip(r1.point_at(u), r2.point_at(u)))

In [8]:
for u in ti:
    print r1.point_at(u)

(0, 1, 0)
(0, -1, 0)


## Curvatura

In [9]:
kappa = r1.curvature().simplify(); kappa

sqrt(2)/(sin(_t)**2 + 1)**(3/2)

In [10]:
solve(kappa.diff(t),t)

[0, pi/2, pi, 3*pi/2]

In [11]:
set(1/kappa.subs(t,u) for u in _)

{sqrt(2)/2, 2}

## Punt més allunyat

### Per Lagrange

In [12]:
l,m = symbols('lambda mu')

In [13]:
L = x**2 + y**2 + z**2 + sum(g*(e.lhs - e.rhs) for g,e in zip((l,m), eqs))

In [14]:
from sympy import diff, solve
leqs = [Eq(diff(L,v).factor()) for v in (x,y,z,l,m)]; leqs

[2*x*(lambda + 1) == 0,
 2*y*(lambda + mu + 1) == 0,
 2*z*(mu + 1) == 0,
 x**2 + y**2 - 1 == 0,
 y**2 + z**2 - 1 == 0]

In [15]:
s = solve(leqs[0]); s

[{lambda: -1}, {x: 0}]

In [16]:
a=s[1]
bs = solve([e.subs(a) for e in leqs][-2:]); bs

[{y: -1, z: 0}, {y: 1, z: 0}]

In [17]:
b=bs[-1]
print [e.subs(a).subs(b) for e in leqs]
Matrix([x,y,z]).subs(a).subs(b)

[True, 2*lambda + 2*mu + 2 == 0, True, True, True]


Matrix([
[0],
[1],
[0]])

In [18]:
a.update(solve(_[-1], dict=True))

In [19]:
b = ct2[1]; b

NameError: name 'ct2' is not defined

In [ ]:
[e.subs(a).subs(b) for e in Out[53]]

In [ ]:
ct3 = solve([e for e in _ if e != True]); ct3

In [ ]:
from sympy import Matrix
for c in ct3:
    assert all(e.subs(a).subs(b).subs(c) for e in Out[53])
    print Matrix([x,y,z]).subs(a).subs(b).subs(c)

In [ ]:
cs = solve([e for e in _ if e != True])

In [ ]:
[e.subs(cs[3]) for e in Out[53]]

In [ ]:
a

### Directament

In [ ]:
(r1.point_at(t).norm()**2).simplify()

In [ ]:
Eq(diff(_,t))

In [ ]:
for u in solve(_,t):
    print r1.point_at(u).norm(), u, r1.point_at(u)

# Problema 2

In [33]:
import sympy
from sympy import symbols, Eq, solve, sin, cos, pi, Matrix
x,y,z = symbols('x y z', real=True)
r,a = symbols('r a', real=True)
from sympy import diff
l = symbols('l', real=True)
f = 6 + x**2 - y**2 + 2*x*y

## Punt més alt

In [34]:
L = f - l*(x**2 + y**2 - 4)
eqs = [Eq(diff(L,c).factor()) for c in (x,y,l)]
eqs

[-2*(l*x - x - y) == 0, 2*(-l*y + x - y) == 0, -x**2 - y**2 + 4 == 0]

In [35]:
for s in solve(eqs):
    print float(f.subs(s))

0.343145750508
0.343145750508
11.6568542495
11.6568542495


### Amb parametrització

In [37]:
S = {x: 2*cos(a), y: 2*sin(a), z: z}

#### A l'interior

In [38]:
Matrix([diff(f,u) for u in (x,y)])

Matrix([
[2*x + 2*y],
[2*x - 2*y]])

In [39]:
solve(_,(x,y))

{y: 0, x: 0}

In [40]:
f.subs(_)

6

#### A la frontera

In [48]:
f.subs(S).simplify()

4*sqrt(2)*sin(2*a + pi/4) + 6

In [49]:
_.diff(a)

8*sqrt(2)*cos(2*a + pi/4)

In [50]:
solve(_,a)

[pi/8, 5*pi/8]

In [51]:
_ + [c + pi for c in _ if c+pi < 2*pi]

[pi/8, 5*pi/8, 9*pi/8, 13*pi/8]

In [52]:
for u in _:
    print Matrix([x,y]).subs(S).subs(a,u), float(f.subs(S).subs(a,u))

Matrix([[2*sqrt(sqrt(2)/4 + 1/2)], [2*sqrt(-sqrt(2)/4 + 1/2)]]) 11.6568542495
Matrix([[-2*sqrt(-sqrt(2)/4 + 1/2)], [2*sqrt(sqrt(2)/4 + 1/2)]]) 0.343145750508
Matrix([[-2*sqrt(sqrt(2)/4 + 1/2)], [-2*sqrt(-sqrt(2)/4 + 1/2)]]) 11.6568542495
Matrix([[2*sqrt(-sqrt(2)/4 + 1/2)], [-2*sqrt(sqrt(2)/4 + 1/2)]]) 0.343145750508


In [53]:
f.subs(S).subs(a,pi/8)

2*sqrt(2) + 8*sqrt(-sqrt(2)/4 + 1/2)*sqrt(sqrt(2)/4 + 1/2) + 6

In [54]:
float(_)

11.65685424949238

## Àrea de la coberta

In [55]:
from sympy import integrate
R ={x: r*cos(a), y: r*sin(a), z: f}
R.update({z: R[z].subs(R).simplify()})
R

{y: r*sin(a), x: r*cos(a), z: sqrt(2)*r**2*sin(2*a + pi/4) + 6}

In [56]:
Rr = Matrix([diff(c.subs(R), r) for c in (x,y,z)])
Ra = Matrix([diff(c.subs(R), a) for c in (x,y,z)])

In [57]:
from sympy import powdenest
dR = powdenest(Rr.cross(Ra)).simplify(); print dR
dR.norm().simplify()

Matrix([
[-2*sqrt(2)*r**2*sin(a + pi/4)],
[-2*sqrt(2)*r**2*cos(a + pi/4)],
[                            r]])


sqrt(8*r**2 + 1)*Abs(r)

In [58]:
integrate(integrate(_, (r,0,2)), (a,0,2*pi))

2*pi*Integral(sqrt(8*r**2 + 1)*Abs(r), (r, 0, 2))

In [59]:
float(_)

49.36765908543893

#### Amb rectangulars

In [ ]:
Matrix([-diff(f,x), -diff(f,y), 1])

In [ ]:
_.norm().simplify()

In [ ]:
_.subs({x: r*cos(a), y: r*sin(a)}).simplify()

In [ ]:
integrate(integrate(_*r, (r,0,2)), (a,0,2*pi))

## Àrea lateral

In [ ]:
Sa = Matrix([diff(c.subs(S), a) for c in (x,y,z)])
Sz = Matrix([diff(c.subs(S), z) for c in (x,y,z)])

In [ ]:
dS = Sa.cross(Sz); print dS
dS.norm().simplify()

In [ ]:
zmax = f.subs(S); zmax.simplify()

In [ ]:
integrate(Out[73], (z,0,zmax))

In [ ]:
areaL = integrate(_, (a,0,2*pi)); areaL

## Flux de la coberta

### Directament

In [60]:
F = Matrix([x,y,z])

In [61]:
F.subs(R)

Matrix([
[                        r*cos(a)],
[                        r*sin(a)],
[sqrt(2)*r**2*sin(2*a + pi/4) + 6]])

In [62]:
dR.simplify()

Matrix([
[-2*sqrt(2)*r**2*sin(a + pi/4)],
[-2*sqrt(2)*r**2*cos(a + pi/4)],
[                            r]])

In [ ]:
F.dot(dR).simplify()

In [ ]:
integrate(_, (r,0,2))

In [ ]:
integrate(_, (a,0,2*pi))

In [ ]:
F.subs(R)

In [ ]:
_.dot(dR)

In [ ]:
_.simplify()

In [ ]:
integrate(_, (r,0,2))

In [ ]:
integrate(_, (a,0,2*pi))

#### Amb cartesianes

In [ ]:
XY = Matrix([x,y,z]).subs(z,f)

In [ ]:
XYx = Matrix([diff(c, x) for c in XY]); print XYx
XYy = Matrix([diff(c, y) for c in XY]); print XYy
XYx.cross(XYy)

In [ ]:
F.subs(z,f).dot(_).simplify()

In [ ]:
(_.subs({x: r*cos(a), y: r*sin(a)})*r)

In [ ]:
integrate(_, (r,0,2))

In [ ]:
integrate(_, (a,0,2*pi))

### Usant el teorema de la divergència

#### Part de la divergència

In [ ]:
divF = sum(diff(c,u) for c,u in zip(F,[x,y,z])); divF

In [ ]:
integrate(divF, (z,0,f))

In [ ]:
_.subs({x: r*cos(a), y:r*sin(a)}).simplify()

In [ ]:
integrate(_*r, (r,0,2))

In [ ]:
Fdiv = integrate(_, (a,0,2*pi)); Fdiv

#### Part de l'area lateral

In [ ]:
F.subs(S).dot(dS).simplify()

In [ ]:
integrate(_, (z,0,f.subs(S)))

In [ ]:
FL = integrate(_, (a,0,2*pi)); FL

#### Tapa inferior

In [ ]:
T = {x: r*cos(a), y: r*sin(a), z: 0}
Tr = Matrix([diff(c.subs(T),r) for c in (x,y,z)])
Ta = Matrix([diff(c.subs(T),a) for c in (x,y,z)])
dT = Tr.cross(Ta)
dT.simplify(); print dT
FI = F.subs(T).dot(dT); FI

In [ ]:
Fdiv - FL + FI

In [ ]:
from sympy.plotting import plot3d_parametric_surface

In [ ]:
w = symbols('w', real=True)

In [ ]:
pS = plot3d_parametric_surface(x.subs(R), y.subs(R), z.subs(R), (r,0,2), (a,0,2*pi))

In [ ]:
pL = plot3d_parametric_surface(x.subs(S), y.subs(S), w*f.subs(S), (a,0,2*pi), (w,0,1))

In [ ]:
pS.extend(pL)

In [ ]:
pS.show()

In [ ]:
z